# Paper tutorial

In [ ]:
import pybinding as pb
import matplotlib.pyplot as plt
import numpy as np
import scipy
import os
import shutil
import subprocess
import kite
from matplotlib import gridspec

pb.pltutils.use_style()

# Haldane model

In [ ]:
from math import sqrt

a = 0.24595  # [nm] unit cell length
a_cc = 0.142 # [nm] carbon-carbon distance
def haldane(t=1, t_nn=1 / 3, phi=0.,delta=0.):
    """Return the lattice specification for Haldane model"""

    a = a_cc * sqrt(3)
    lat = pb.Lattice(a1=[a, 0], a2=[a / 2, a / 2 * sqrt(3)])

    # The next-nearest hoppings shift the Dirac point away from zero energy.
    # This will push it back to zero for consistency with the first-nearest model.

    lat.add_sublattices(
        ('A', [0, -a_cc / 2], delta),
        ('B', [0, a_cc / 2], -delta)
    )

    lat.add_hoppings(
        ([0, 0], 'A', 'B', t),
        ([1, -1], 'A', 'B', t),
        ([0, -1], 'A', 'B', t),
        ([0, 1], 'A', 'A', t_nn * np.exp(-1j * phi)),
        ([0, 1], 'B', 'B', t_nn * np.exp(1j * phi)),
        ([1, -1], 'A', 'A', t_nn * np.exp(-1j * phi)),
        ([1, -1], 'B', 'B', t_nn * np.exp(1j * phi)),
        ([1, 0], 'A', 'A', t_nn * np.exp(1j * phi)),
        ([1, 0], 'B', 'B', t_nn * np.exp(-1j * phi))
    )

    return lat


# make a haldane lattice
t = 1
t_nn = 1/3
from math import pi
lattice = haldane(t=t, t_nn=t_nn, phi=pi * 0.5, delta=2 * t_nn)
# add Uniformly distributed disorder
disorder = kite.Disorder(lattice)
dis_std = 0.4
disorder.add_disorder('A', 'Uniform', +0.0, dis_std)
disorder.add_disorder('B', 'Uniform', +0.0, dis_std)
# number of decomposition parts in each direction of matrix.
# This divides the lattice into various sections, each of which is calculated in parallel
domain_decompose_1 = 2
domain_decompose_2 = 1
# number of unit cells in each direction.
lx = 256
ly = 256

num_moments = 256
configuration = kite.Configuration(divisions=[domain_decompose_1, domain_decompose_2],
                                   length=[lx, ly], boundaries=[True, True],
                                   is_complex=True, precision=0, spectrum_range=[-3.8, 3.8])
calculation = kite.Calculation(configuration)

# require the calculation of DOS and conductivity_dc
direction = 'xy'
num_random = 1
calculation.dos(num_points=1000, num_moments=num_moments, num_random=num_random, num_disorder=1)
calculation.conductivity_dc(num_points=1000, num_moments=num_moments, num_random=num_random, num_disorder=1,
                            direction=direction, temperature=1)
# configure the *.h5 file
name = 'haldane_topol_{}_dis_std_{}_moments_{}_random_{}_lx_{}_ly_{}.h5'
full_name = name.format(direction, dis_std, num_moments, num_random, lx, ly)
kite.config_system(lattice, configuration, calculation, disorder=disorder, filename=full_name)

In [ ]:
pb.utils.tic()  # measure the time
kite.KITEx(full_name)
pb.utils.toc('Time for moments calculation ')

pb.utils.tic()  # measure the time
kite.KITEtools(full_name + ' --CondDC -T 50 -F -3 3 10000 -E 4000 -K green 0.01 -N condDC_haldane_{}.dat'.format(direction))
kite.KITEtools(full_name + ' --DOS -F -3 3 100 -K green 0.01 -N dos_haldane.dat')
pb.utils.toc('Time for reconstructing ')

In [ ]:
cond_raw = np.loadtxt('condDC_haldane_{}.dat'.format(direction))
dos_raw = np.loadtxt('dos_haldane.dat')

plt.figure()
ax1 = plt.gca()
ax1.plot(cond_raw[:,0], cond_raw[:,1])
ax1.set_xlabel(r'$\mathrm{E~(eV)}$')
ax1.set_ylabel(r'$\mathrm{\sigma_{xx}~(e^2/h)}$')

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
ax2.plot(dos_raw[:,0], dos_raw[:,1], c='C1')
plt.sca(ax2)
ax2.tick_params(axis='y', labelcolor='C1')
ax2.set_ylabel(r'$\mathrm{DOS~(a.u.)}$')